Cookies & Session & SSO & OAuth
===

介绍一下网站开发中，和登陆相关的各个知识点。

简要介绍原理和实现方法，可以在开发工作中灵活运用。

## Cookies

### 简介

在 HTTP 0.9 时代，HTTP 是一个无状态协议，这对如何保持用户状态产生了困难。

所以 Netscape 公司在 HTTP 中增加了 cookies，这是一个简单的字符串，以键值对的形式保存数据。



### 作用域

cookies 以 `USER@DOMAIN:PORT` 的形式保存在浏览器内。也就是说，改变上述值中的任意一部分，都会导致产生一个新的 cookie 。

比如 `chexiang.com` 和 `chexiang.com:8080` 就是两个不同的域，cookies 互相隔离。

### 用法

![null](http://laisky-blog.qiniudn.com/cookies.png)

一般来讲，流程如下：

  0. 客户端访问服务器（登陆或普通访问）
  0. 服务器响应，并给客户端 set_cookies，用于纪录用户状态
  0. 客户端之后的请求都会自动包含这段 cookies，服务器可以借以识别用户

### 实战：登录

一个标准的登陆流程如下：

  0. 用户 POST 用户名 & 密码
  0. 服务器验证密码，验证成功，返回 302 & set_cookies(username=xxx)
  0. 用户被 302 跳转（到首页），然后请求网页
  0. 服务器验证用户的 cookie，验证成功后返回页面（视为登录成功）

### 属性

cookies 是一个字符串，由若干的 cookie 以 `;` 连接起来，一个完整的 cookie 包含以下属性：

  - `name`：名字；
  - `value`：值；
  - `domain`：如 `.chexiang.com` 就会对 `xxx.chexiang.com` 都生效；
  - `path`：指定生效的 URL，一般指定为 `/`；
  - `age`：生效时间，设定一个秒数；
  - `expires`：过期时间，指定一个日期；
  - `httponly`：只允许 HTTP 访问 cookie，而不允许 Javascript 访问；
  - `secure`：只允许“安全的”传输 cookie；
  - `hostonly`：是否严格检验 domain。

### 使用注意

网站开发面向的是用户，负责传递前后台的数据交互，要时刻谨记安全性问题，头脑里对于数据的安全性要有一个深刻的认识。

简单的说，凡事用户发来的数据，都是不安全的。

不难理解，cookies 也是一个不安全的数据。


cookies 的一些特点：

  - 不安全，用户可伪造任意 cookie；（使用加密 cookie 串）；
  - 不稳定，用户随时可能删掉 cookie；
  - 有大小限制，一般来说总 cookies 大小不超过 20 M（实际应用中就不应该上 MB）；
  - 性能很差，因为是字符串，每次操作都是遍历；
  - 无法跨于，使用场景有限。


## Session

当年，为了解决 cookies 的一系列问题，人们推出了 session（会话）。

和 cookies 是 HTTP 标准不同，session 只是一种人们为解决某个问题而想出的标准方法，并没有标准可言。

不过 session 在现代架构设计里已经不再推荐使用，所以我只简单介绍下。

### 简介

cookies 最大的问题是安全性，也就是说我们不能在 cookies 里存放任何敏感数据，而只能存一些能表征用户身份的信息。

但是一个用户难免会有些敏感数据，这些数据属于状态，修改频繁，不方便存放在用户库里，为此 session 应运而生。

session 是一个抽象的概念，泛指用户的一次会话过程，也就是一个身份确认的用户的整个生命周期（从登录到注销）。

### 实现

![null](http://laisky-blog.qiniudn.com/session.png)

会发现 Session 和 Cookies 最大的区别就是增加了一个后端存储。而且 session 的视线也依赖于 cookies，不过这次 cookies 里不再存任何数据，而仅仅存一个 SessionId 即可。

其实在仔细想想，session 和 cookies 的仅仅在于**是否根据 SID 来存放一份安全的会话数据**。

后端数据库并不是必要的，如果你对你的加密协议有信心的话，将会话数据加密存放在 cookies 里也是可以的。

### 优点

  - 极大的简化 cookies，不再存放大量数据，而仅仅需要存放一个 SID，然后在后端可以存放任意数量的用户状态数据
  - 永久有效，基于 UID 和 SID 识别用户，不再依赖于浏览器_？_


### 缺点

  - 分布式的噩梦
    - 难以跨机器共享
    - 难以保持一致性
    - 难以跨集群共享

这时候你会觉得，session 能做到的好像 cookie 都能做到嘛，而且我 cookie 天生对集群友好。

不管你有多少集群多少后端节点，我浏览器都一视同仁。

这也是现在推荐使用 cookie 而不是 session 的原因。

_当然你要装逼，硬上分布式 session 架构也是可以的🙃，还可以制造很多工作岗位_

## cookies 加密

顺带扯一下 cookies 加密。

我个人偏好使用 https://jwt.io 这个协议。

可以使用密钥（证书）将对象（object, dict）转变一个 token 字符串，服务端可以使用相同的密钥对 token 进行检验，确保用户无法伪造 token。

同时，token 内保存的数据保持，任何人都可以取用。

也就是：“你可以看，但是你没法改”，拿来放 UID 简直理想。

## Single Sign-On(SSO)

名字挺高端，就是俗称的单点登录

![null](http://laisky-blog.qiniudn.com/sso.png)

关键是要理解登录的本质是什么，登录只是用户的一个**状态**，而这个状态我们用 cookie 来存储。

所以，要实现在 xx 网站登录，就是要在 xx 网站上写入一段 cookie。

由此可见，核心问题变成了**跨域**，也就是如何跨域写入 cookie。

所以 SSO 可以被拆解为两个问题：

  - 同域名网站的 cookie；
  - 不同域名网站的 cookie。

### 同域名网站

比如 `a.a.com` 和 `b.a.com`，这种很简单，只要把 cookies 的 domain 设置为 `.a.com` 即可。


### 不同域名网站

这种稍微麻烦点，不过也有一整套的成熟技术，比如：

  - CORS：通过设置 `Access-Control-Allow-Origin` header 来允许跨域访问；
  - jsonp：通过 jsonp 来跨域设置 cookies。
  
简单介绍下 jsonp，我在 a 网站通过 jsonp 的方式调用 b.com/set_cookie.js?cookie=xxx 就实现了在 b.com 内写入指定 cookie。

## OAuth

OAuth 是现在跨应用跨站点间授权的标准化方式。

比如你常见的“通过新浪微博登陆”、“同步到微博”、“通过微信登录”、“通过 OpenID 登陆”等等等等，都是通过 OAuth 实现的。

### 角色

要清楚，在一个完整的 OAuth 认证流程中，至少存在以下三个角色：

  - 用户
  - 应用服务器：给用户提供服务的服务器，可以有多台
  - 认证服务器：负责认证授权的服务器

### 流程

![null](http://laisky-blog.qiniudn.com/oauth.png)

### 优点

  - 安全性：用户的所有数据都保存在授权服务器，应用仅能拿到用户的 UID
  - 可控性：用户在每次授权的时候都可以选择给应用什么权限，而且可以随时吊销 token
  - 适用性：几乎可实现你任何的跨站认证需求
  - 唯一性：授权服务器负责维护用户数据，给全应用平台提供唯一的 UID
  
### 缺点

  - 稍微有点复杂…
  - 需要来回跳转，认证速度较慢（但是认证成功后就很快乐）

## Q&A